<a href="https://colab.research.google.com/github/MunJinSeo/MyProject/blob/master/daniel021_nsmc_koelectra_base_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어(NSMC) KoELECTRA를_이용한_감정분석기: Colab(Pytorch + HuggingFace)
# Github : https://github.com/MunJinSeo/MyProject/
<br>

## References 1
- (1) 김희규님의 "HuggingFace KoElectra로 NSMC 감성분석 Fine-tuning해보기"<br>
https://heegyukim.medium.com/huggingface-koelectra%EB%A1%9C-nsmc-%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0-1a23a0c704af

- (2) 이지원님의 Github : nlp_emotion_classification <br>
https://github.com/jiwonny/nlp_emotion_classification

## 사용모델 KoELECTRA
- 한국어 : 박장원님의 KoELECTRA 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset (학습에는 train만 사용함, test는 감점대상)
- 한국어 : 네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc <br>
ratings_train.txt <br>
ratings_test.txt

## 과제 파일
- https://www.kaggle.com/c/korean-sa-competition-bdc101 <br>
ko_data.csv

## References 2
- https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP
- https://blog.naver.com/horajjan/221739630055
<br>@전처리 관련@<br>
- https://github.com/YongWookHa/kor-text-preprocess
- https://github.com/likejazz/korean-sentence-splitter
- https://github.com/lovit/soynlp
<br>@@<br>
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249


## 실행 방법 / 유의사항
- Colab 에서 반드시 GPU로 실행 (Colab Pro 권장)
- 과제파일(ko_data.csv)은 사전에 직접 호스팅 서버로 업로드 해줘야함
- 모델 학습을 위한 NSMC 데이터셋은 자동 다운로드 처리됨 (train만 사용함)
- 소스는 위에서부터 순차적으로 실행하면 됨
- CUDA(GPU) 메모리 오버되는경우 학습시 Batch size 줄여서 해볼것 -- 다만, 너무 작으면 성능 떨어짐
- 만약 정확도 확인하는 부문에서 CUDA(GPU) 메모리 오버가 나오면 ▷ 모델 별도 저장 ▷ 런타임 초기화 ▷ 모델 로딩 ▷ 학습은 건너뛰고 정확도 실행
- KoELECTRA base_v3으로 사용했으며 1epoch 당 약 60분

## 처리 순서 
- 필요 lib 설치
- NSMC 데이터셋 다운로드
- 필요 모듈 import
- 데이터셋 처리 (Dataset Calss / 전처리)
- 모델 생성 (Create Model)
- 학습(Learn) - train파일만 사용(test제외)
- 테스트 데이터셋 정확도 확인하기
- 모델 저장하기
- 과제용 데이터 예측 및 맵핑
- 결과 파일 저장

##--------------------------------------------------------------------


# 필요 lib 설치

In [ ]:
# lib 설치
!pip install transformers
!pip install torch

In [ ]:
!pip install kss
!pip install konlpy
!pip install sentencepiece
!pip install soynlp

In [ ]:
# (미사용) Colab TPU 사용을 위해 설치
#--!pip install torch_xla
#--#@param ["1.5" , "20200325", "nightly"]
#VERSION = "1.7"
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION

# NSMC 데이터셋 다운로드

In [ ]:
#NSMC 데이터셋 다운로드

#!git clone https://github.com/e9t/nsmc.git

# 학습에는 train만 사용함, test는 감점대상
#!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

# ko_data.csv #별도 kaggle에서 받아서 사용 https://www.kaggle.com/c/korean-sa-competition-bdc101

--2020-12-20 13:26:09--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt.1’

ratings_train.txt.1 100%[===================>]  13.95M  37.3MB/s    in 0.4s    

2020-12-20 13:26:10 (37.3 MB/s) - ‘ratings_train.txt.1’ saved [14628807/14628807]

--2020-12-20 13:26:10--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: 

In [ ]:
#!head ratings.txt
!head ratings_train.txt
!head ratings_test.txt
# !head ko_data.csv #별도 kaggle에서 받아서 사용 https://www.kaggle.com/c/korean-sa-competition-bdc101

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


# 필요 모듈 import

In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# (미사용) TPU 사용을 위해 필요
#import torch_xla
#import torch_xla.core.xla_model as xm

In [ ]:
import kss
import re
from soynlp.normalizer import *

In [ ]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

# (미사용) TPU
#device = xm.xla_device()

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# 데이터셋 처리 (Dataset Calss / 전처리)
(train data와 과제 sample data형식이 다르고, encoding이 다르기 때문에 분리 처리)

In [ ]:
class NSMC_Dataset(Dataset):
  
  def __init__(self, csv_file, ftype):
    # train data와 sample data 각각 처리

    # 초기 전처리 1, 아래쪽 전처리 2 로 clean_text() 펑션 분리함 (학습 할때 line별 전처리 후 사용됨)
    # --- 전처리 1 start ------------------------

    if ftype == 'train':
      # 일부 값중에 NaN 제거
      self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0)

    elif ftype == 'sample':
      # 한글처리용 cp949적용
      self.dataset = pd.read_csv(csv_file, sep=',', encoding='cp949') 
       # 뒤쪽 컬럼 label 값이 없으므로 기본값으로 추가함
      self.dataset.insert(2,'Predicted','-')
      #print(self.dataset)
    else:
      self.dataset = pd.read_csv(csv_file, sep=',').dropna(axis=0)


    # for idx, document in self.dataset.iterrows():
    #   print(idx, document)
    self.sub1 = re.compile('[^ .?!/@$%~|0-9|ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기, 특수기호 일부를 제외한 모든 글자제거
    self.sub2 = re.compile('[\s]+')  # white space duplicate
    self.sub3 = re.compile('[\.]+')  # full stop duplicate

    #중복되는 문장 제거
    if ftype == 'sample':
      print('과제 제출용은 중복 문장 제거하면 안됨')
    else:
      # 중복제거: document가 리뷰 텍스트 내용의 title명이다
      self.dataset.drop_duplicates(subset=['document'], inplace=True)

    # dataset 확인
    print(self.dataset.describe())
    print(self.dataset)

    # tokenizer
    #self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
    self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    # 문장 분리하여 일정 길이 넘는 것만 사용
    ttStr = ""
    for ssStr in kss.split_sentences(cleaned):
      #print(ssStr)
      if len(ssStr) > 1:
        #ttStr += ssStr + " \n"
        ttStr += "[CLS] " + ssStr + " [SEP]"

    cleaned = ttStr
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):
    # 행번호별 컬럼 지정하여 할당
    row = self.dataset.iloc[idx, 1:3].values  # idx번째 행의 첫번째 컬럼 0을 제외하고 1~3컬럼 할당
    text = self.clean_text( txt=row[0] ) # 전처리 2 : clean_text()
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
#학습에는 train만 사용함, test는 감점대상
train_dataset = NSMC_Dataset("ratings_train.txt","train")
test_dataset = NSMC_Dataset("ratings_test.txt","train")
sample_dataset = NSMC_Dataset("ko_data.csv","sample")

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9

In [ ]:
tmpstr = '훌륭하다. 초한지 얼른 읽어보고 다시 봐야겠다. 연출 훌륭하다 껄껄 한신의 토사구팽은 슬펐다'
print( train_dataset.clean_text( txt = tmpstr) )

[CLS] 훌륭하다. [SEP][CLS] 초한지 얼른 읽어보고 다시 봐야겠다. [SEP][CLS] 연출 훌륭하다 [SEP][CLS] 껄껄 한신의 토사구팽은 슬펐다 [SEP]


# 모델 생성 (Create Model)

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
model.cuda()

# 한번 실행해보기
#text, attention_mask, y = train_dataset[0]
#model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
try:
  model.load_state_dict(torch.load("model.pt"))
except:
  print("error - model.load_state_dict(torch.load('model.pt'))")
else:
  print("success - model.load_state_dict(torch.load('model.pt'))")

error - model.load_state_dict(torch.load('model.pt'))


In [ ]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# 학습(Learn) - train파일만 사용(test제외)

In [ ]:
# koelectra-small-v3-discriminator
#epochs = 16
#batch_size = 128

# koelectra-base-v3-discriminator
epochs = 12
batch_size = 32 #CUDA(GPU) 사용량 오버되면 Batch size를 줄여서 해볼것, 단,너무 작으면 성능 떨어짐

In [ ]:
optimizer = AdamW(model.parameters(), 
                  lr=1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  )
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
losses = []
accuracies = []

# 그래디언트 초기화
# model.zero_grad()

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  # 훈련모드로 변경
  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward() # Backward 수행으로 그래디언트 계산
    #xm.optimizer_step(optimizer, barrier=True)  # TPU 사용시 코드
    optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
    #model.zero_grad() # 그래디언트 초기화

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 62.62380960583687 Accuracy: tensor(0.6587, device='cuda:0')
Batch Loss: 102.92454545199871 Accuracy: tensor(0.7455, device='cuda:0')
Batch Loss: 139.20437735319138 Accuracy: tensor(0.7812, device='cuda:0')
Batch Loss: 172.60216480493546 Accuracy: tensor(0.8003, device='cuda:0')
Batch Loss: 205.57341362535954 Accuracy: tensor(0.8116, device='cuda:0')
Batch Loss: 236.4764873534441 Accuracy: tensor(0.8221, device='cuda:0')
Batch Loss: 268.31255979835987 Accuracy: tensor(0.8286, device='cuda:0')
Batch Loss: 297.6803382188082 Accuracy: tensor(0.8345, device='cuda:0')
Batch Loss: 327.97140665352345 Accuracy: tensor(0.8389, device='cuda:0')
Batch Loss: 356.15520618855953 Accuracy: tensor(0.8431, device='cuda:0')
Batch Loss: 384.95520489662886 Accuracy: tensor(0.8464, device='cuda:0')
Batch Loss: 413.6491841673851 Accuracy: tensor(0.8491, device='cuda:0')
Batch Loss: 441.09680350869894 Accuracy: tensor(0.8521, device='cuda:0')
Batch Loss: 469.38213277608156 Accuracy: tensor(0.8541,

Batch Loss: 22.122401475906372 Accuracy: tensor(0.9122, device='cuda:0')
Batch Loss: 42.48413397744298 Accuracy: tensor(0.9153, device='cuda:0')
Batch Loss: 63.13858676701784 Accuracy: tensor(0.9166, device='cuda:0')
Batch Loss: 85.2905140966177 Accuracy: tensor(0.9141, device='cuda:0')
Batch Loss: 105.43504744023085 Accuracy: tensor(0.9153, device='cuda:0')
Batch Loss: 126.22060088813305 Accuracy: tensor(0.9152, device='cuda:0')
Batch Loss: 148.29360474646091 Accuracy: tensor(0.9136, device='cuda:0')
Batch Loss: 170.25645140185952 Accuracy: tensor(0.9133, device='cuda:0')
Batch Loss: 190.66802613064647 Accuracy: tensor(0.9132, device='cuda:0')
Batch Loss: 210.99080598540604 Accuracy: tensor(0.9137, device='cuda:0')
Batch Loss: 231.76572968624532 Accuracy: tensor(0.9138, device='cuda:0')
Batch Loss: 255.3171674516052 Accuracy: tensor(0.9131, device='cuda:0')
Batch Loss: 275.14282331056893 Accuracy: tensor(0.9137, device='cuda:0')
Batch Loss: 297.14248730428517 Accuracy: tensor(0.9137, 

Batch Loss: 17.976485040038824 Accuracy: tensor(0.9341, device='cuda:0')
Batch Loss: 35.05241291038692 Accuracy: tensor(0.9352, device='cuda:0')
Batch Loss: 51.12208195589483 Accuracy: tensor(0.9354, device='cuda:0')
Batch Loss: 66.0821052249521 Accuracy: tensor(0.9367, device='cuda:0')
Batch Loss: 83.45604174025357 Accuracy: tensor(0.9358, device='cuda:0')
Batch Loss: 100.61670267023146 Accuracy: tensor(0.9354, device='cuda:0')
Batch Loss: 118.27672975324094 Accuracy: tensor(0.9342, device='cuda:0')
Batch Loss: 134.97632236219943 Accuracy: tensor(0.9341, device='cuda:0')
Batch Loss: 150.70006732456386 Accuracy: tensor(0.9346, device='cuda:0')
Batch Loss: 167.8397638015449 Accuracy: tensor(0.9347, device='cuda:0')
Batch Loss: 185.1348059028387 Accuracy: tensor(0.9342, device='cuda:0')
Batch Loss: 200.7726102527231 Accuracy: tensor(0.9346, device='cuda:0')
Batch Loss: 218.27392490394413 Accuracy: tensor(0.9344, device='cuda:0')
Batch Loss: 234.18923201598227 Accuracy: tensor(0.9346, dev

Batch Loss: 11.020855291746557 Accuracy: tensor(0.9628, device='cuda:0')
Batch Loss: 23.552957522682846 Accuracy: tensor(0.9561, device='cuda:0')
Batch Loss: 36.15100650023669 Accuracy: tensor(0.9550, device='cuda:0')
Batch Loss: 49.77968760486692 Accuracy: tensor(0.9538, device='cuda:0')
Batch Loss: 62.01629238482565 Accuracy: tensor(0.9543, device='cuda:0')
Batch Loss: 74.33777006994933 Accuracy: tensor(0.9543, device='cuda:0')
Batch Loss: 86.19888997077942 Accuracy: tensor(0.9548, device='cuda:0')
Batch Loss: 97.64508725237101 Accuracy: tensor(0.9551, device='cuda:0')
Batch Loss: 110.76472462993115 Accuracy: tensor(0.9542, device='cuda:0')
Batch Loss: 123.94547300320119 Accuracy: tensor(0.9538, device='cuda:0')
Batch Loss: 137.30559353437275 Accuracy: tensor(0.9537, device='cuda:0')
Batch Loss: 150.38996005430818 Accuracy: tensor(0.9535, device='cuda:0')
Batch Loss: 163.4992881361395 Accuracy: tensor(0.9532, device='cuda:0')
Batch Loss: 177.3879498038441 Accuracy: tensor(0.9527, dev

...

Batch Loss: 4.737340489518829 Accuracy: tensor(0.9850, device='cuda:0')
Batch Loss: 8.779380915570073 Accuracy: tensor(0.9856, device='cuda:0')
Batch Loss: 14.60887099802494 Accuracy: tensor(0.9838, device='cuda:0')
Batch Loss: 20.242886436521076 Accuracy: tensor(0.9834, device='cuda:0')
Batch Loss: 25.696834185509942 Accuracy: tensor(0.9829, device='cuda:0')
Batch Loss: 30.704506149399094 Accuracy: tensor(0.9828, device='cuda:0')
Batch Loss: 36.69765545881819 Accuracy: tensor(0.9826, device='cuda:0')
Batch Loss: 42.72391129878815 Accuracy: tensor(0.9821, device='cuda:0')
Batch Loss: 47.80874137498904 Accuracy: tensor(0.9823, device='cuda:0')
Batch Loss: 52.779040044290014 Accuracy: tensor(0.9823, device='cuda:0')
Batch Loss: 56.76107079174835 Accuracy: tensor(0.9827, device='cuda:0')
Batch Loss: 63.306989686447196 Accuracy: tensor(0.9823, device='cuda:0')
Batch Loss: 68.05614119546954 Accuracy: tensor(0.9823, device='cuda:0')
Batch Loss: 72.7495283872122 Accuracy: tensor(0.9825, devic

Batch Loss: 4.036162015981972 Accuracy: tensor(0.9831, device='cuda:0')
Batch Loss: 8.982439295272343 Accuracy: tensor(0.9836, device='cuda:0')
Batch Loss: 13.670815063873306 Accuracy: tensor(0.9828, device='cuda:0')
Batch Loss: 17.273620884749107 Accuracy: tensor(0.9840, device='cuda:0')
Batch Loss: 22.264407496899366 Accuracy: tensor(0.9838, device='cuda:0')
Batch Loss: 25.90274058864452 Accuracy: tensor(0.9842, device='cuda:0')
Batch Loss: 29.306922382034827 Accuracy: tensor(0.9845, device='cuda:0')
Batch Loss: 33.68030328885652 Accuracy: tensor(0.9845, device='cuda:0')
Batch Loss: 38.12526290817186 Accuracy: tensor(0.9846, device='cuda:0')
Batch Loss: 42.70528900285717 Accuracy: tensor(0.9845, device='cuda:0')
Batch Loss: 46.808810713584535 Accuracy: tensor(0.9849, device='cuda:0')
Batch Loss: 52.27023501053918 Accuracy: tensor(0.9846, device='cuda:0')
Batch Loss: 56.939178482745774 Accuracy: tensor(0.9846, device='cuda:0')
Batch Loss: 61.37660714366939 Accuracy: tensor(0.9845, dev

Batch Loss: 4.317599073052406 Accuracy: tensor(0.9859, device='cuda:0')
Batch Loss: 8.021300944499671 Accuracy: tensor(0.9873, device='cuda:0')
Batch Loss: 12.783888153848238 Accuracy: tensor(0.9860, device='cuda:0')
Batch Loss: 16.424085068516433 Accuracy: tensor(0.9863, device='cuda:0')
Batch Loss: 20.137036147993058 Accuracy: tensor(0.9864, device='cuda:0')
Batch Loss: 24.98973730043508 Accuracy: tensor(0.9860, device='cuda:0')
Batch Loss: 29.241253098181915 Accuracy: tensor(0.9859, device='cuda:0')
Batch Loss: 33.75844857265474 Accuracy: tensor(0.9855, device='cuda:0')
Batch Loss: 38.0398265235126 Accuracy: tensor(0.9857, device='cuda:0')
Batch Loss: 41.437727162847295 Accuracy: tensor(0.9858, device='cuda:0')
Batch Loss: 44.935303549980745 Accuracy: tensor(0.9861, device='cuda:0')
Batch Loss: 48.00846659473609 Accuracy: tensor(0.9861, device='cuda:0')
Batch Loss: 52.37433689692989 Accuracy: tensor(0.9861, device='cuda:0')
Batch Loss: 56.621459301095456 Accuracy: tensor(0.9858, dev

Batch Loss: 3.6091167867416516 Accuracy: tensor(0.9862, device='cuda:0')
Batch Loss: 6.856628111447208 Accuracy: tensor(0.9873, device='cuda:0')
Batch Loss: 9.352713285945356 Accuracy: tensor(0.9890, device='cuda:0')
Batch Loss: 12.937212617311161 Accuracy: tensor(0.9888, device='cuda:0')
Batch Loss: 16.94012738694437 Accuracy: tensor(0.9879, device='cuda:0')
Batch Loss: 21.02827177918516 Accuracy: tensor(0.9878, device='cuda:0')
Batch Loss: 24.567541823955253 Accuracy: tensor(0.9876, device='cuda:0')
Batch Loss: 28.042290340992622 Accuracy: tensor(0.9876, device='cuda:0')
Batch Loss: 31.957818921771832 Accuracy: tensor(0.9874, device='cuda:0')
Batch Loss: 35.07087428076193 Accuracy: tensor(0.9874, device='cuda:0')
Batch Loss: 38.18169368535746 Accuracy: tensor(0.9875, device='cuda:0')
Batch Loss: 41.26233234204119 Accuracy: tensor(0.9876, device='cuda:0')
Batch Loss: 45.58101819670992 Accuracy: tensor(0.9874, device='cuda:0')
Batch Loss: 49.396348230249714 Accuracy: tensor(0.9872, dev

Batch Loss: 3.5504452987224795 Accuracy: tensor(0.9869, device='cuda:0')
Batch Loss: 6.4369383148732595 Accuracy: tensor(0.9869, device='cuda:0')
Batch Loss: 8.768047972640488 Accuracy: tensor(0.9891, device='cuda:0')
Batch Loss: 12.28915962157771 Accuracy: tensor(0.9891, device='cuda:0')
Batch Loss: 15.25233791518258 Accuracy: tensor(0.9894, device='cuda:0')
Batch Loss: 18.177162798296195 Accuracy: tensor(0.9897, device='cuda:0')
Batch Loss: 21.738232628791593 Accuracy: tensor(0.9893, device='cuda:0')
Batch Loss: 25.442749101202935 Accuracy: tensor(0.9890, device='cuda:0')
Batch Loss: 28.547017387405504 Accuracy: tensor(0.9891, device='cuda:0')
Batch Loss: 31.729678622505162 Accuracy: tensor(0.9889, device='cuda:0')
Batch Loss: 34.82699480006704 Accuracy: tensor(0.9889, device='cuda:0')
Batch Loss: 38.85388565721223 Accuracy: tensor(0.9885, device='cuda:0')
Batch Loss: 42.488163320638705 Accuracy: tensor(0.9884, device='cuda:0')
Batch Loss: 45.89435382868396 Accuracy: tensor(0.9884, d

In [ ]:
losses, accuracies

([1294.1683600954711,
  948.5586515292525,
  769.478010000661,
  612.5274829221889,
  477.91572357015684,
  371.3163122392725,
  306.9550270232139,
  256.01489251770545,
  218.35722130234353,
  193.64528291486204,
  168.77805006943527,
  155.0497188034351],
 [tensor(0.8803, device='cuda:0'),
  tensor(0.9165, device='cuda:0'),
  tensor(0.9339, device='cuda:0'),
  tensor(0.9493, device='cuda:0'),
  tensor(0.9619, device='cuda:0'),
  tensor(0.9708, device='cuda:0'),
  tensor(0.9763, device='cuda:0'),
  tensor(0.9804, device='cuda:0'),
  tensor(0.9834, device='cuda:0'),
  tensor(0.9851, device='cuda:0'),
  tensor(0.9867, device='cuda:0'),
  tensor(0.9880, device='cuda:0')])

# 테스트 데이터셋 정확도 확인하기

In [ ]:
# 평가모드로 변경
model.eval()

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



Accuracy: tensor(0.9043, device='cuda:0')


# 모델 저장하기

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")

# 과제용 데이터 예측 및 맵핑

In [ ]:
#과제용 데이터 예측
# 데이터 로딩
batchSize = 16
sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

#평가모드로 변경
model.eval()

idx_s = 0
idx_e = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)

  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  global idx_s, idx_e
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  #print("index==", idx_s, idx_e)
  #print("--start-- sample_result['predict_label'][idx_s : idx_e]==\n" , sample_result['predict_label'][idx_s : idx_e] )
  sample_result['Predicted'][idx_s : idx_e] = rsList  #배치구간을 한번에 업데이트
  #print("--end-- sample_result['predict_label'][idx_s : idx_e]==\n" , sample_result['predict_label'][idx_s : idx_e] )
  idx_s += len(rsList) #해당 배치구간내에 index 시작값은 윗줄 처리 후 증가

  #test_correct += (predicted == y_batch).sum()
  #test_total += len(y_batch)

# print("Accuracy:", test_correct.float() / test_total)
print(sample_result)


          Id                                           Sentence Predicted
0          0                                   정말 많이 울었던 영화입니다.         -
1          1                                           시간 낭비예요.         -
2          2             포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.         -
3          3               지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!         -
4          4                          이걸 영화로 만드는 거야?얼마나 가는지 보자.         -
...      ...                                                ...       ...
11182  11182  이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...         -
11183  11183                                     심심__ 그냥 한효주 cf         -
11184  11184  공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...         -
11185  11185                                      오토바이 신은 최고네요.         -
11186  11186                                   개병헌 쓰면 엉망이 된다ㅋㅋㅋ         -

[11187 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



          Id                                           Sentence Predicted
0          0                                   정말 많이 울었던 영화입니다.         1
1          1                                           시간 낭비예요.         0
2          2             포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.         1
3          3               지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!         1
4          4                          이걸 영화로 만드는 거야?얼마나 가는지 보자.         0
...      ...                                                ...       ...
11182  11182  이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...         1
11183  11183                                     심심__ 그냥 한효주 cf         0
11184  11184  공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...         1
11185  11185                                      오토바이 신은 최고네요.         1
11186  11186                                   개병헌 쓰면 엉망이 된다ㅋㅋㅋ         0

[11187 rows x 3 columns]


In [ ]:
#torch.cuda.empty_cache() #GPU 캐쉬 데이터 삭제

# 결과 파일 저장

In [ ]:
# 주어진 데이터의 결과를 파일로 저장
#sample_csv = sample_result.to_csv('sample_ko.csv')
sample_csv = sample_result.to_csv('sample_ko.csv',sep=',',na_rep='NaN', columns=['Id','Predicted'],index=False)

In [ ]:
# 파일을 PC로 다운로드 하기
from google.colab import files
files.download('sample_ko.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 나의 구글Drive 연결
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive
